<a href="https://colab.research.google.com/github/natitedros/WAVN-Federated-Learning/blob/main/CNN_Homing_FL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

Mounted at /content/drive


In [4]:
CSV_PATH = '/content/drive/MyDrive/gazebo_dataset_fl/labels.csv'
IMG_DIR = '/content/drive/MyDrive/gazebo_dataset_fl/images/'

In [5]:
df = pd.read_csv(CSV_PATH)
print(f"Total samples: {len(df)}")
print(f"\nFirst few rows:\n{df.head()}")
print(f"\nDirection counts:\n{df['direction'].value_counts()}")

Total samples: 60

First few rows:
      current_image     destination_image direction
0  0001_current.png  0001_destination.png   forward
1  0002_current.png  0002_destination.png  backward
2  0003_current.png  0003_destination.png     right
3  0004_current.png  0004_destination.png   forward
4  0005_current.png  0005_destination.png     right

Direction counts:
direction
forward     22
left        18
right       12
backward     8
Name: count, dtype: int64


In [7]:
direction_map = {'forward': 0, 'backward': 1, 'left': 2, 'right': 3}
df['direction_label'] = df['direction'].map(direction_map)
num_clients = 4
shard_size = len(df) // num_clients

In [18]:
client_datasets = []
for i in range(num_clients):
    start = i * shard_size
    end = start + shard_size
    shard = df.iloc[start:end]

    # For each shard create train/val split
    train_df, val_df = train_test_split(shard, test_size=0.2, random_state=42)

    client_datasets.append((train_df, val_df))
    print(f"\nClient: {i+1}")
    print(f"Training samples: {len(train_df)}")
    print(f"Validation samples: {len(val_df)}")


Client: 1
Training samples: 12
Validation samples: 3

Client: 2
Training samples: 12
Validation samples: 3

Client: 3
Training samples: 12
Validation samples: 3

Client: 4
Training samples: 12
Validation samples: 3


In [19]:
# Function to load image
def load_image(img_path):
    img = load_img(IMG_DIR + img_path, target_size=(224, 224))
    img = img_to_array(img) / 255.0  # Normalize to [0, 1]
    return img

In [20]:
# Creating the dataset
def create_dataset(dataframe, batch_size=32):
    current_images = []
    dest_images = []
    labels = []

    for idx, row in dataframe.iterrows():
        current_img = load_image(row['current_image'])
        dest_img = load_image(row['destination_image'])

        current_images.append(current_img)
        dest_images.append(dest_img)
        labels.append(row['direction_label'])

    current_images = np.array(current_images)
    dest_images = np.array(dest_images)
    labels = np.array(labels)

    return current_images, dest_images, labels

In [21]:
for i, (train_df, val_df) in enumerate(client_datasets):

  print(f"\nClient: {i+1}")
  print("Loading training data...")
  X_train_current, X_train_dest, y_train = create_dataset(train_df)

  print("Loading validation data...")
  X_val_current, X_val_dest, y_val = create_dataset(val_df)

  print(f"\nTraining data shapes:")
  print(f"Current images: {X_train_current.shape}")
  print(f"Destination images: {X_train_dest.shape}")
  print(f"Labels: {y_train.shape}")


Client: 1
Loading training data...
Loading validation data...

Training data shapes:
Current images: (12, 224, 224, 3)
Destination images: (12, 224, 224, 3)
Labels: (12,)

Client: 2
Loading training data...
Loading validation data...

Training data shapes:
Current images: (12, 224, 224, 3)
Destination images: (12, 224, 224, 3)
Labels: (12,)

Client: 3
Loading training data...
Loading validation data...

Training data shapes:
Current images: (12, 224, 224, 3)
Destination images: (12, 224, 224, 3)
Labels: (12,)

Client: 4
Loading training data...
Loading validation data...

Training data shapes:
Current images: (12, 224, 224, 3)
Destination images: (12, 224, 224, 3)
Labels: (12,)


In [22]:
from tensorflow.keras import layers, models

def create_model(num_classes=4):
    # Input for current image
    current_input = layers.Input(shape=(224, 224, 3), name='current_image')

    # Input for destination image
    dest_input = layers.Input(shape=(224, 224, 3), name='destination_image')

    # Shared CNN layers
    def cnn_branch(x):
        x = layers.Conv2D(32, (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
        x = layers.Conv2D(64, (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
        x = layers.Conv2D(64, (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D((2, 2))(x)
        x = layers.Flatten()(x)
        return x

    # Process both images
    current_features = cnn_branch(current_input)
    dest_features = cnn_branch(dest_input)

    # Combine features
    combined = layers.concatenate([current_features, dest_features])

    # Dense layers
    x = layers.Dense(128, activation='relu')(combined)
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(num_classes, activation='softmax')(x)

    # Create model
    model = models.Model(inputs=[current_input, dest_input], outputs=output)

    return model

In [24]:
model = []
for i in range(num_clients):
    model.append(create_model(num_classes=4))
model[0].summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ current_image       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ destination_image   │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 222, 222,  │        896 │ current_image[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 222, 222,  │        896 │ destination_imag… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_6     │ (None, 111, 111,  │          0 │ conv2d_6[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_9     │ (None, 111, 111,  │          0 │ conv2d_9[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 109, 109,  │     18,496 │ max_pooling2d_6[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 109, 109,  │     18,496 │ max_pooling2d_9[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_7     │ (None, 54, 54,    │          0 │ conv2d_7[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_10    │ (None, 54, 54,    │          0 │ conv2d_10[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 52, 52,    │     36,928 │ max_pooling2d_7[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 52, 52,    │     36,928 │ max_pooling2d_10… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_8     │ (None, 26, 26,    │          0 │ conv2d_8[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_11    │ (None, 26, 26,    │          0 │ conv2d_11[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 43264)     │          0 │ max_pooling2d_8[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 43264)     │          0 │ max_pooling2d_11… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 86528)     │          0 │ flatten_2[0][0],  │
│ (Concatenate)       │                   │            │ flatten_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 11,188,868 (42.68 MB)

 Trainable params: 11,188,868 (42.68 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
for i in range(num_clients):
    model[i].compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

In [26]:
# Training the model
# batch size is the number of data instances to train at once before updating the weights
# not related with pixel size. Usually assigned powers of 2 numbers for hardware efficiency
for i in range(num_clients):
    print(f"\nTraining client {i+1}...")
    history = model[i].fit(
        [X_train_current, X_train_dest],
        y_train,
        validation_data=([X_val_current, X_val_dest], y_val),
        epochs=5,
        batch_size=32
    )


Training client 1...
Epoch 1/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 0.0833 - loss: 1.4435 - val_accuracy: 0.0000e+00 - val_loss: 3.4992
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.2500 - loss: 3.4553 - val_accuracy: 0.0000e+00 - val_loss: 1.1933
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.4167 - loss: 2.2961 - val_accuracy: 1.0000 - val_loss: 0.9068
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.3333 - loss: 1.5459 - val_accuracy: 1.0000 - val_loss: 1.2723
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.5000 - loss: 1.2708 - val_accuracy: 0.0000e+00 - val_loss: 1.4636

Training client 2...
Epoch 1/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.5833 - loss: 1.3091 - val_accuracy: 0.0000e+00 - val_loss: 8.7049
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.4167 - loss: 5.4121 - val_accuracy: 1.0000 - val_loss: 0.0799
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.1667 - loss: 4.5166 - val_

In [27]:
# Getting the weights
w0 = model[0].get_weights()
w1 = model[1].get_weights()
w2 = model[2].get_weights()
w3 = model[3].get_weights()

print(f"w0: {len(w0)} arrays with shapes: {[w.shape for w in w0]}")
print(f"w1: {len(w1)} arrays with shapes: {[w.shape for w in w1]}")
print(f"w2: {len(w2)} arrays with shapes: {[w.shape for w in w2]}")
print(f"w3: {len(w3)} arrays with shapes: {[w.shape for w in w3]}")

pairwise_dist = np.array([w0, w1, w2, w3], dtype=object)

w0: 16 arrays with shapes: [(3, 3, 3, 32), (32,), (3, 3, 3, 32), (32,), (3, 3, 32, 64), (64,), (3, 3, 32, 64), (64,), (3, 3, 64, 64), (64,), (3, 3, 64, 64), (64,), (86528, 128), (128,), (128, 4), (4,)]
w1: 16 arrays with shapes: [(3, 3, 3, 32), (32,), (3, 3, 3, 32), (32,), (3, 3, 32, 64), (64,), (3, 3, 32, 64), (64,), (3, 3, 64, 64), (64,), (3, 3, 64, 64), (64,), (86528, 128), (128,), (128, 4), (4,)]
w2: 16 arrays with shapes: [(3, 3, 3, 32), (32,), (3, 3, 3, 32), (32,), (3, 3, 32, 64), (64,), (3, 3, 32, 64), (64,), (3, 3, 64, 64), (64,), (3, 3, 64, 64), (64,), (86528, 128), (128,), (128, 4), (4,)]
w3: 16 arrays with shapes: [(3, 3, 3, 32), (32,), (3, 3, 3, 32), (32,), (3, 3, 32, 64), (64,), (3, 3, 32, 64), (64,), (3, 3, 64, 64), (64,), (3, 3, 64, 64), (64,), (86528, 128), (128,), (128, 4), (4,)]


In [29]:
# Performing the federated averaging
avg_weights = np.average(pairwise_dist, axis=0)
print(f"Average Weight: {len(avg_weights)} arrays with shapes: {[w.shape for w in avg_weights]}")

Average Weight: 16 arrays with shapes: [(3, 3, 3, 32), (32,), (3, 3, 3, 32), (32,), (3, 3, 32, 64), (64,), (3, 3, 32, 64), (64,), (3, 3, 64, 64), (64,), (3, 3, 64, 64), (64,), (86528, 128), (128,), (128, 4), (4,)]


In [30]:
# Create a new model with the same architecture as model[0]
avg_model = create_model()

avg_model.set_weights(avg_weights)

In [31]:
# Save the model
avg_model.save('/content/drive/MyDrive/Models/navigation_model_fl.h5')
print("\nModel saved to Google Drive!")


Model saved to Google Drive!


In [32]:
# Step 14: Function to test specific image pairs
def test_prediction(current_img_name, dest_img_name):
    """
    Test prediction for specific images by name
    Using the average model
    Args:
        current_img_name: Name of current image (e.g., 'current_001.jpg')
        dest_img_name: Name of destination image (e.g., 'destination_015.jpg')
    """
    direction_names = {0: 'forward', 1: 'backward', 2: 'left', 3: 'right'}

    # Load the images
    current_img = load_image(current_img_name)
    dest_img = load_image(dest_img_name)

    # Add batch dimension
    current_img = np.expand_dims(current_img, axis=0)
    dest_img = np.expand_dims(dest_img, axis=0)

    # Make prediction
    prediction = avg_model.predict([current_img, dest_img], verbose=0)
    predicted_label = np.argmax(prediction)
    confidence = prediction[0][predicted_label] * 100

    print(f"\n{'='*50}")
    print(f"Current image: {current_img_name}")
    print(f"Destination image: {dest_img_name}")
    print(f"Predicted direction: {direction_names[predicted_label]}")
    print(f"Confidence: {confidence:.2f}%")
    print(f"\nAll probabilities:")
    for label, prob in enumerate(prediction[0]):
        print(f"  {direction_names[label]}: {prob*100:.2f}%")
    print(f"{'='*50}")

    return direction_names[predicted_label]

In [33]:
test_prediction('0002_destination.png', '0007_destination.png')


Current image: 0002_destination.png
Destination image: 0007_destination.png
Predicted direction: forward
Confidence: 25.38%

All probabilities:
  forward: 25.38%
  backward: 25.01%
  left: 24.85%
  right: 24.76%


'forward'